In [ ]:
import cv2
from glob import glob
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv3D,MaxPool3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
sea_fish=pathlib.Path("/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited")

In [ ]:
zero_list=list(sea_fish.glob("Crabs/*.jpg"))
one_list=list(sea_fish.glob("Dolphin/*.jpg"))
two_list=list(sea_fish.glob("Eel/*.jpg"))
three_list=list(sea_fish.glob("Jelly Fish/*.jpg"))
four_list=list(sea_fish.glob("Lobster/*.jpg"))
five_list=list(sea_fish.glob("Octopus/*.jpg"))
six_list=list(sea_fish.glob("Penguin/*.jpg"))
seven_list=list(sea_fish.glob("Seahorse/*.jpg"))
eight_list=list(sea_fish.glob("Starfish/*.jpg"))
nine_list=list(sea_fish.glob("Turtle_Tortoise/*.jpg"))
ten_list=list(sea_fish.glob("Whale/*.jpg"))



In [ ]:
len(zero_list),len(one_list),len(two_list),len(three_list),len(four_list),len(five_list),len(six_list),len(seven_list),len(eight_list),len(nine_list),len(ten_list)

(200, 223, 195, 216, 217, 210, 202, 206, 223, 212, 215)

In [ ]:
fish_dict={"Crabs":zero_list,
               "Dolphin":one_list,
               "Eel":two_list,
               "Jelly Fish":three_list,
               "Lobster":four_list,
               "Octopus":five_list,
               "Penguin":six_list,
               "Seahorse":seven_list,
               "Starfish":eight_list,
               "Turtle_Tortoise":nine_list,
               "Whale":ten_list}

fish_class={"Crabs":0,
               "Dolphin":1,
               "Eel":2,
               "Jelly Fish":3,
               "Lobster":4,
               "Octopus":5,
               "Penguin":6,
               "Seahorse":7,
               "Starfish":8,
               "Turtle_Tortoise":9,
               "Whale":10}

In [ ]:
x=[]
y=[]

In [ ]:
for i in fish_dict:
  fish_name=i
  sea_fish_list=fish_dict[fish_name]
  for path in sea_fish_list:
    img=cv2.imread(str(path))
    img=cv2.resize(img,(224,224))
    img=img/255
    x.append(img)
    cls=fish_class[fish_name]
    y.append(cls)

In [ ]:
len(x)

2319

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.75,random_state=1)

In [ ]:
len(x_train),len(y_train),len(x_test),len(y_test)

(1739, 1739, 580, 580)

In [ ]:
x_train.shape

(1739, 224, 224, 3)

In [ ]:
x_train.shape,x_test.shape

((1739, 224, 224, 3), (580, 224, 224, 3))

Changes From CNN to Transfer Learning starts from here

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2


In [ ]:
base_model=tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                              include_top=False,
                                              weights='imagenet')

In [ ]:
print("[INFO] summary for base model...")
print(base_model.summary())

[INFO] summary for base model...
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )             

In [ ]:
from tensorflow.keras.layers import MaxPooling2D
from keras.layers.core import Dropout

from tensorflow.keras.models import Model
headModel= base_model.output
headModel=MaxPooling2D(pool_size=(2,2))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(64,activation="relu")(headModel)
headModel=Dropout(0.2)(headModel)
headModel=Dense(11,activation="softmax")(headModel)
model=Model(inputs=base_model.input,outputs=headModel)
for layer in base_model.layers:
  layer.trainable=False

# These are commands of CNN(Down)

# from tensorflow.keras import datasets, layers, models
# model=models.Sequential()
# model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(224,224,3)))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Conv2D(64, (3,3), activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Conv2D(64, (3,3), activation='relu'))
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(11, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam
print("[INFO] compiling model...")
opt=Adam(lr=1e-4)

model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])
print("[INFO] training head...")

model_hist=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test),batch_size=180)

[INFO] compiling model...
[INFO] training head...
Epoch 1/10
10/10 [==============================] - 124s 13s/step - loss: 3.9783 - accuracy: 0.2835 - val_loss: 1.7510 - val_accuracy: 0.3690
Epoch 2/10
10/10 [==============================] - 110s 11s/step - loss: 1.5462 - accuracy: 0.4405 - val_loss: 1.4814 - val_accuracy: 0.5069
Epoch 3/10
10/10 [==============================] - 109s 11s/step - loss: 1.2514 - accuracy: 0.5612 - val_loss: 1.2929 - val_accuracy: 0.5862
Epoch 4/10
10/10 [==============================] - 93s 10s/step - loss: 1.0161 - accuracy: 0.6371 - val_loss: 1.1165 - val_accuracy: 0.6552
Epoch 5/10
10/10 [==============================] - 112s 11s/step - loss: 0.8060 - accuracy: 0.7044 - val_loss: 1.0511 - val_accuracy: 0.7052
Epoch 6/10
10/10 [==============================] - 100s 10s/step - loss: 0.7485 - accuracy: 0.7280 - val_loss: 0.9931 - val_accuracy: 0.7103
Epoch 7/10
10/10 [==============================] - 120s 13s/step - loss: 0.6645 - accuracy: 0.7660

In [ ]:
model.save("model.h3")


In [ ]:
from tensorflow.keras.preprocessing import image
def testing_image(image_directory):
  test_image=image.load_img(image_directory, target_size=(224,224))
  test_image=image.img_to_array(test_image)
  test_image=np.expand_dims(test_image, axis=0)
  test_image=test_image/255
  result=model.predict(x=test_image)
  print(result)
  if np.argmax(result)== 0:
    prediction= 'Crab'
  elif np.argmax(result)== 1:
    prediction= 'Dolphin'
  elif np.argmax(result)== 2:
    prediction= 'Eel'
  elif np.argmax(result)== 3:
    prediction= 'Jelly Fish'
  elif np.argmax(result)== 4:
    prediction= 'Lobster'
  elif np.argmax(result)== 5:
    prediction= 'Octopus'
  elif np.argmax(result)== 6:
    prediction= 'Penguin'
  elif np.argmax(result)== 7:
    prediction= 'Seahorse'
  elif np.argmax(result)== 8:
    prediction= 'Starfish'
  elif np.argmax(result)== 9:
    prediction= 'Turtle_Tortoise'
  elif np.argmax(result)== 10:
    prediction= 'Whale'

  return prediction

In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Crabs/14305505763_cf0c04a4c9_o.jpg'))

1/1 [==============================] - 0s 70ms/step
[[9.6283561e-01 5.3883489e-04 1.7471981e-06 2.4576588e-03 2.6668259e-04
  5.8859034e-04 6.3174048e-05 6.2156742e-04 7.5344404e-05 3.2515239e-02
  3.5609664e-05]]
Crab


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Dolphin/6079914961_73bfeecbf9_o.jpg'))

1/1 [==============================] - 0s 64ms/step
[[6.5462464e-06 9.7399139e-01 6.0903393e-09 2.0466525e-04 9.9568488e-04
  2.0712973e-04 4.7162521e-05 4.2362753e-08 1.8657272e-07 1.2763100e-07
  2.4547070e-02]]
Dolphin


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Eel/13170307045_a6c776c632_o.jpg'))

1/1 [==============================] - 0s 173ms/step
[[0.04478999 0.05784832 0.25088528 0.01566708 0.05116455 0.06863502
  0.07993509 0.10406996 0.10815796 0.05090693 0.16793986]]
Eel


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Jelly Fish/14331959343_45b748f229_b.jpg'))

1/1 [==============================] - 0s 69ms/step
[[2.0601080e-07 2.7072962e-04 5.1743097e-08 9.9838281e-01 5.1348592e-04
  2.9827427e-07 1.2691273e-08 1.2628360e-06 4.3878208e-12 1.7636111e-06
  8.2928198e-04]]
Jelly Fish


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Lobster/13964394414_c15dc672fc_b.jpg'))

1/1 [==============================] - 0s 63ms/step
[[1.72150764e-03 1.46199707e-02 2.74057221e-03 4.39169295e-02
  4.45964009e-01 3.58709954e-02 1.82846081e-04 2.20527992e-01
  1.05086627e-04 1.13661557e-01 1.20688505e-01]]
Lobster


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Octopus/12186301654_e7c8a9e89c_o.jpg'))

1/1 [==============================] - 0s 106ms/step
[[0.01477304 0.09650148 0.03727651 0.0008848  0.02058157 0.36791903
  0.01582278 0.11623484 0.03464645 0.05809611 0.23726343]]
Octopus


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Penguin/13217360313_e50902b6ee_o.jpg'))

1/1 [==============================] - 0s 170ms/step
[[4.72717889e-04 8.81493613e-02 2.07080971e-02 2.80442350e-06
  4.45490468e-06 4.50854888e-04 6.51791036e-01 3.90474615e-03
  1.19876482e-01 1.09043058e-05 1.14628606e-01]]
Penguin


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Seahorse/15942875240_8b9242e742_b.jpg'))

1/1 [==============================] - 0s 64ms/step
[[8.6573988e-07 3.5015272e-04 2.9715005e-05 3.2534599e-02 1.1776456e-02
  2.8776651e-04 4.8207451e-04 9.5289099e-01 8.9986443e-06 1.4264284e-03
  2.1194748e-04]]
Seahorse


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Starfish/14308001574_2576e36830_b.jpg'))

1/1 [==============================] - 0s 68ms/step
[[1.5022373e-07 4.3684607e-09 1.1621721e-06 2.4639718e-10 5.7856187e-06
  1.1021608e-09 1.6268204e-05 9.7811281e-08 9.9995470e-01 2.1761540e-05
  3.7366515e-08]]
Starfish


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Turtle_Tortoise/Turtle_Tortoise (1084).jpg'))

1/1 [==============================] - 0s 111ms/step
[[6.9429370e-06 2.4892573e-04 7.2621560e-07 5.4515176e-06 7.0578908e-04
  2.0784562e-04 3.8061976e-06 1.9969370e-03 2.8292058e-04 9.9652988e-01
  1.0699460e-05]]
Turtle_Tortoise


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Whale/12524370965_7a1c01eab6_o.jpg'))

1/1 [==============================] - 0s 123ms/step
[[1.89318566e-03 5.47491275e-02 4.27161540e-05 8.06549549e-01
  3.25008482e-02 5.21976850e-04 1.09090855e-04 3.22430948e-04
  3.98576822e-06 9.56084055e-04 1.02351092e-01]]
Jelly Fish


In [ ]:
print(testing_image('/content/drive/MyDrive/DL Datasets/Sea Fish Dataset/Sea Fishes Dataset Edited/Crabs/14305505763_cf0c04a4c9_o.jpg'))

1/1 [==============================] - 0s 104ms/step
[[9.6283561e-01 5.3883489e-04 1.7471981e-06 2.4576588e-03 2.6668259e-04
  5.8859034e-04 6.3174048e-05 6.2156742e-04 7.5344404e-05 3.2515239e-02
  3.5609664e-05]]
Crab
